In [1]:
data_var = '2023-12-05'

In [2]:
import pandas as pd; pd.set_option('display.max_columns', None)
from datetime import date, timedelta
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [3]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [4]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [5]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [6]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [7]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
2119,2023-12-05,Brasil Nbb,20:00,Corinthians Paulista,Mogi,1.28,3.44,154.5,1.85,1.85,-9.5,1.98,1.74,dIQHWh0K,0.781250,0.290698,0.540541,0.540541,0.071948,0.0,0.0,NaN,NaN,151.768,26.258312,0.173016,227.814,143.962601,0.631930,0.000,0.0,136.30,130.40,0.0,0.0,0.0,0.0,0.647182,0.000000,0.091240,0.00,0.000,inf,0.0,0.0,0.0,0.0,0.0,inf,0.0,0.0,0.0
2120,2023-12-05,Brasil Nbb,20:00,Paulistano,Flamengo,2.50,1.49,145.5,1.85,1.83,2.5,2.10,1.65,IRRDXYpE,0.400000,0.671141,0.540541,0.546448,0.071141,0.6,0.0,NaN,NaN,125.006,37.006913,0.296041,110.676,31.328007,0.283061,137.062,0.0,166.14,106.40,0.0,0.0,0.0,0.0,0.357984,0.007686,0.169706,2.13,0.426,3.521127,0.0,0.0,0.0,0.0,0.0,inf,0.0,0.0,0.0
2121,2023-12-05,Eua Torneio Nba In,00:00,Sacramento Kings,New Orleans Pelicans,1.70,2.34,236.5,1.83,2.05,-3.5,2.05,1.89,296nlizK,0.588235,0.427350,0.546448,0.487805,0.015586,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.224034,0.080187,0.057430,0.00,0.000,inf,0.0,0.0,0.0,0.0,0.0,inf,0.0,0.0,0.0
2122,2023-12-05,Eua Torneio Nba In,21:30,Milwaukee Bucks,New York Knicks,1.53,2.76,224.5,1.82,2.11,-5.5,2.06,1.87,CvSokk57,0.653595,0.362319,0.549451,0.473934,0.015914,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.405474,0.104357,0.068372,0.00,0.000,inf,0.0,0.0,0.0,0.0,0.0,inf,0.0,0.0,0.0
2123,2023-12-05,Europa Euroliga,14:30,Anadolu Efes,Panathinaikos,2.88,1.45,163.5,1.87,1.95,4.5,2.00,1.71,xxGqsFze,0.347222,0.689655,0.534759,0.512821,0.036877,0.0,0.0,NaN,NaN,188.710,58.675693,0.310930,143.778,38.136134,0.265243,0.000,0.0,167.00,202.71,0.0,0.0,0.0,0.0,0.467050,0.029617,0.110545,0.00,0.000,inf,0.0,0.0,0.0,0.0,0.0,inf,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2199,2023-12-05,Europa Copa Europeia Da Fiba,16:00,Zaragoza,Nymburk,1.26,3.69,164.5,1.85,1.87,-10.5,2.06,1.66,tIKB0wrf,0.793651,0.271003,0.540541,0.534759,0.064654,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.0,79.52,110.58,0.0,0.0,0.0,0.0,0.694250,0.007603,0.152066,0.00,0.000,inf,0.0,0.0,0.0,0.0,0.0,inf,0.0,0.0,0.0
2200,2023-12-05,França Copa Da França,15:00,Avenir Chartres F,Flammes Carolo F,6.04,1.10,140.5,1.85,1.85,12.5,1.95,1.74,C41y2eq0,0.165563,0.909091,0.540541,0.540541,0.074654,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.978461,0.000000,0.080484,0.00,0.000,inf,0.0,0.0,0.0,0.0,0.0,inf,0.0,0.0,0.0
2201,2023-12-05,Israel Superliga,14:00,Galil Elyon,Nes Ziona,6.16,1.11,170.5,1.83,1.91,11.5,2.10,1.62,lGXyIPXH,0.162338,0.900901,0.546448,0.523560,0.063239,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.0,0.00,150.96,0.0,0.0,0.0,0.0,0.982363,0.030251,0.182479,0.00,0.000,inf,0.0,0.0,0.0,0.0,0.0,inf,0.0,0.0,0.0
2202,2023-12-05,Israel Superliga,16:00,Maccabi Ironi Ramat Gan,Hapoel Jerusalem,1.17,4.88,160.5,1.80,1.92,-12.5,2.00,1.70,UNpkDtoc,0.854701,0.204918,0.555556,0.520833,0.059619,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.0,0.00,145.35,0.0,0.0,0.0,0.0,0.867228,0.045620,0.114666,0.00,0.000,inf,0.0,0.0,0.0,0.0,0.0,inf,0.0,0.0,0.0


## Apostas do dia

### Over v0 (LDA)

In [8]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
2,00:00,Eua Torneio Nba In,Sacramento Kings,New Orleans Pelicans,236.5,1.83,1.0,Over
3,21:30,Eua Torneio Nba In,Milwaukee Bucks,New York Knicks,224.5,1.82,1.0,Over
6,16:15,Europa Euroliga,Olympiakos,Real Madrid,159.5,1.80,1.0,Over
